In [1]:
!pip install -q sentence-transformers faiss-cpu openai langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.3 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai
import os


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
# ✅ STEP 1: Install required packages
!pip install -q transformers sentence-transformers faiss-cpu

# ✅ STEP 2: Import libraries
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# ✅ STEP 3: Load models
embedder = SentenceTransformer("all-MiniLM-L6-v2")
generator = pipeline(model="google/flan-t5-base", task="text2text-generation")

# ✅ STEP 4: Create a small knowledge base (mock startup-related documents)
docs = [
    "Startup India offers tax exemptions for early-stage startups.",
    "AngelList provides access to investors and startup job seekers.",
    "SIDBI offers funding options for MSMEs through schemes like PRISM.",
    "A business model canvas includes value proposition, revenue streams, and key partners.",
    "Crunchbase has data on startup competitors and funding history.",
    "Incubators like T-Hub and Y Combinator support early startups with mentoring and seed funding.",
    "Legal compliance for Indian startups includes GST registration, MSME registration, and DPIIT recognition.",
    "Go-to-market strategies may involve social media marketing, partnerships, and early adopter outreach."
]

# ✅ STEP 5: Embed documents and build FAISS index
doc_embeddings = embedder.encode(docs, convert_to_tensor=False)
doc_embeddings = np.array(doc_embeddings).astype("float32")
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# ✅ STEP 6: Define the blueprint generator
def generate_blueprint(user_idea, k=3):
    query_embedding = embedder.encode([user_idea])[0].astype("float32")
    _, I = index.search(np.array([query_embedding]), k)
    retrieved_docs = [docs[i] for i in I[0]]

    # Create context for generation
    context = "\n".join(retrieved_docs)
    prompt = f"""Startup Idea: {user_idea}

Context:
{context}

Generate a startup blueprint including:
- Business Model Canvas
- Market Size Estimation
- Legal & Licensing Requirements
- Go-to-Market Strategy
- Funding Avenues
- Estimated Budget & Timeline
"""
    result = generator(prompt, max_new_tokens=512)[0]["generated_text"]
    return result

# ✅ STEP 7: Run the generator with your idea
user_input = "An app that connects rural patients with certified doctors through telemedicine."
blueprint = generate_blueprint(user_input)

# ✅ STEP 8: Print the result
print("📘 Generated Startup Blueprint:\n")
print(blueprint)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


📘 Generated Startup Blueprint:

Go-to-Market Strategy
